In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

### Load signals

In [2]:
df = pd.read_csv('./data/wind-farm-1-signals-2016.csv', sep=';')
df = pd.concat([df, pd.read_csv('./data/wind-farm-1-signals-2017.csv', sep=';')])
print(df.shape)
# date format
df.Timestamp = pd.to_datetime(df.Timestamp.str[:-6])

# assure 10min each
df = df.set_index(['Timestamp']).groupby(['Turbine_ID']).resample('10min').mean().reset_index().sort_values(by=['Turbine_ID','Timestamp'])

df.head()

(417141, 83)


,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Hyd_Oil_Temp_Avg,Gear_Oil_Temp_Avg,Gear_Bear_Temp_Avg,Nac_Temp_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Avg,Amb_WindSpeed_Std,Amb_WindDir_Relative_Avg,Amb_WindDir_Abs_Avg,Amb_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ActPwrGen2,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_ReactPwrGen0,Prod_LatestAvg_ReactPwrGen1,Prod_LatestAvg_ReactPwrGen2,Prod_LatestAvg_TotReactPwr,HVTrafo_Phase1_Temp_Avg,HVTrafo_Phase2_Temp_Avg,HVTrafo_Phase3_Temp_Avg,Grd_InverterPhase1_Temp_Avg,Cont_Top_Temp_Avg,Cont_Hub_Temp_Avg,Cont_VCP_Temp_Avg,Gen_SlipRing_Temp_Avg,Spin_Temp_Avg,Blds_PitchAngle_Min,Blds_PitchAngle_Max,Blds_PitchAngle_Avg,Blds_PitchAngle_Std,Cont_VCP_ChokcoilTemp_Avg,Grd_RtrInvPhase1_Temp_Avg,Grd_RtrInvPhase2_Temp_Avg,Grd_RtrInvPhase3_Temp_Avg,Cont_VCP_WtrTemp_Avg,Grd_Prod_Pwr_Avg,Grd_Prod_CosPhi_Avg,Grd_Prod_Freq_Avg,Grd_Prod_VoltPhse1_Avg,Grd_Prod_VoltPhse2_Avg,Grd_Prod_VoltPhse3_Avg,Grd_Prod_CurPhse1_Avg,Grd_Prod_CurPhse2_Avg,Grd_Prod_CurPhse3_Avg,Grd_Prod_Pwr_Max,Grd_Prod_Pwr_Min,Grd_Busbar_Temp_Avg,Rtr_RPM_Std,Amb_WindSpeed_Est_Avg,Grd_Prod_Pwr_Std,Grd_Prod_ReactPwr_Avg,Grd_Prod_ReactPwr_Max,Grd_Prod_ReactPwr_Min,Grd_Prod_ReactPwr_Std,Grd_Prod_PsblePwr_Avg,Grd_Prod_PsblePwr_Max,Grd_Prod_PsblePwr_Min,Grd_Prod_PsblePwr_Std,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg
0,T01,2016-01-01 00:00:00,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,30.0,44.0,48.0,28.0,11.3,10.9,11.1,11.6,0.5,3.3,0.9,-12.4,206.1,18.0,-107.0,4420.0,0.0,4313.0,-99.0,-5636.0,0.0,-5735.0,68.0,76.0,65.0,39.0,39.0,28.0,43.0,25.0,20.0,-1.1,4.5,0.6,0.9,91.0,39.0,39.0,38.0,39.0,26.2,0.7,50.0,401.6,399.9,399.2,46.5,61.1,45.5,118.4,-33.5,38.0,0.1,3.6,35.3,-34.9,13.2,-100.0,36.2,29.4,119.3,0.0,31.9,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37.0,218.5
1,T01,2016-01-01 00:10:00,1268.3,93.3,999.7,435.9,41.0,58.0,59.0,59.0,30.0,44.0,48.0,28.0,11.2,0.0,8.8,12.5,0.4,3.2,0.9,-1.1,217.4,18.0,-753.0,2488.0,0.0,1735.0,-867.0,-3871.0,0.0,-4738.0,68.0,76.0,65.0,38.0,39.0,28.0,42.0,25.0,20.0,-0.9,59.6,8.1,14.7,91.0,37.0,36.0,37.0,37.0,10.1,0.7,50.0,401.9,400.7,399.5,34.3,41.9,33.6,101.5,-38.2,38.0,4.0,3.3,30.2,-28.0,1.8,-100.1,34.6,21.7,102.3,0.0,24.7,-79.6,0.0,-501.0,123.7,79.6,501.0,0.0,123.7,37.0,218.5
2,T01,2016-01-01 00:20:00,1394.4,220.3,774.0,486.1,41.0,57.0,57.0,57.0,30.0,43.0,46.0,29.0,12.3,1.9,6.8,8.8,0.7,4.3,1.0,-5.8,207.4,18.0,-784.0,10488.0,0.0,9704.0,-410.0,-1865.0,0.0,-2275.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-2.2,24.0,10.4,12.4,87.0,38.0,37.0,37.0,38.0,59.0,0.7,50.0,401.4,400.5,398.8,78.6,92.7,86.1,219.4,-34.1,38.0,4.3,4.1,77.5,-13.9,999.9,-108.1,154.8,88.9,224.9,0.0,54.3,-314.0,0.0,-1000.0,357.4,314.0,1000.0,0.0,357.4,37.0,213.3
3,T01,2016-01-01 00:30:00,1306.6,1212.4,1257.1,17.0,40.0,56.0,57.0,57.0,30.0,44.0,48.0,28.0,11.5,10.7,11.1,9.9,0.7,4.4,1.1,9.6,232.0,18.0,0.0,22673.0,0.0,22673.0,0.0,-15822.0,0.0,-15822.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-1.9,0.9,-0.5,0.8,87.0,38.0,38.0,38.0,38.0,135.7,0.8,50.0,399.7,398.1,396.9,128.1,160.8,136.8,250.3,-27.7,38.0,0.1,4.6,53.9,-94.9,0.0,-105.1,18.4,135.3,261.4,0.0,53.1,-654.0,0.0,-1000.0,245.8,654.0,1000.0,0.0,245.8,36.0,222.4
4,T01,2016-01-01 00:40:00,1320.9,1224.1,1257.7,18.0,40.0,57.0,58.0,57.0,30.0,44.0,48.0,28.0,11.7,10.8,11.1,9.9,0.8,4.1,1.2,23.3,245.6,18.0,-75.0,16581.0,0.0,16506.0,-79.0,-13314.0,0.0,-13394.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-2.0,4.4,-0.1,1.2,88.0,39.0,38.0,38.0,39.0,98.9,0.7,50.0,399.5,397.9,396.8,104.7,130.3,108.3,212.5,-50.2,38.0,0.2,4.3,65.9,-80.3,17.8,-107.1,35.8,102.7,221.8,0.0,60.2,-496.1,0.0,-1000.0,302.7,496.1,1000.0,0.0,302.7,36.0,222.4


In [3]:
df.shape

(421056, 83)

### Load logs

In [25]:
# read logs 2016
df_logs_16 = pd.read_csv('./data/wind-farm-1-logs-2016.csv', sep=';')
df_logs_16 = df_logs_16.rename(columns={'TimeDetected':'Timestamp','TimeReset':'Time_Reset', 'UnitTitle':'Turbine_ID'})

# date format
df_logs_16.Timestamp = pd.to_datetime(df_logs_16.Timestamp.str[:-6])
df_logs_16.Time_Reset = pd.to_datetime(df_logs_16.Time_Reset)

# read logs 2017
df_logs_17 = pd.read_csv('./data/wind-farm-1-logs-2017.csv', sep=';')
df_logs_17 = df_logs_17.rename(columns={'Time_Detected':'Timestamp','Turbine_Identifier':'Turbine_ID'})

# date format
df_logs_17.Timestamp = pd.to_datetime(df_logs_17.Timestamp.str[:-6])
df_logs_17.Time_Reset = pd.to_datetime(df_logs_17.Time_Reset.str[:-6])

# concat
df_logs = pd.concat([df_logs_16, df_logs_17]).dropna(how='all', axis=1).dropna(subset=['Timestamp']).sort_values(by=['Timestamp'])

df_logs.head()

/tmp/ipykernel_1406/3510147951.py:10: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_logs_17 = pd.read_csv('./data/wind-farm-1-logs-2017.csv', sep=';')


,Timestamp,Time_Reset,Turbine_ID,Remark,Unit_Title_Destination
68215,2016-01-01 00:02:18,NaT,T11,External power ref.:2000kW,NaN
21311,2016-01-01 00:07:15,NaT,T06,Generator 1 in,NaN
115209,2016-01-01 00:15:27,NaT,T01,Generator 1 in,NaN
43463,2016-01-01 00:15:44,NaT,T01,External power ref.:2000kW,NaN
115210,2016-01-01 00:17:13,NaT,T07,External power ref.:2000kW,NaN


### Load failures

In [26]:
# read logs 2016
df_fail = pd.concat([
    pd.read_excel('./data/wind-farm-1-failures-2016.xlsx'),
    pd.read_csv('./data/wind-farm-1-failures-2017.csv', sep=';')])

# date format
df_fail.Timestamp = pd.to_datetime(df_fail.Timestamp.str[:-6])

df_fail = df_fail.dropna(subset=['Turbine_ID','Timestamp'], how='any').sort_values(by=['Timestamp'])
df_fail.head()

,Turbine_ID,Component,Timestamp,Remarks
14,T11,GENERATOR,2016-03-03 19:00:00,Electric circuit error in generator
6,T06,HYDRAULIC_GROUP,2016-04-04 18:53:00,Error in pitch regulation
7,T07,GENERATOR_BEARING,2016-04-30 12:40:00,High temperature in generator bearing (replace...
11,T09,GENERATOR_BEARING,2016-06-07 16:59:00,High tempemperature generator bearing
8,T07,TRANSFORMER,2016-07-10 03:46:00,High temperature transformer


### Merge signals & logs
Turbine working has a log 'External power ref: XXXkW' each 40 minutes  
Time_Reset seems to be when the turbine recovers from a pause or error

In [27]:
# concat logs
df_logs_join = df_logs.copy()

# convert to 10 min
df_logs_join.Timestamp = pd.to_datetime(df_logs_join.Timestamp.dt.strftime('%Y-%m-%d %H:%M').str[:-1] + '0:00')
df_logs_join.Time_Reset = pd.to_datetime(df_logs_join.Time_Reset.dt.strftime('%Y-%m-%d %H:%M').str[:-1] + '0:00')
df_logs_join['logs_Remark'] = np.where(df_logs_join.Time_Reset.isna()==False, 'Reset:' + df_logs_join.Time_Reset.dt.strftime('%Y-%m-%d %H:%M:%S'), '') + df_logs_join.Remark.fillna('').astype(str)
df_logs_join['epr'] = np.where(df_logs_join.logs_Remark.str.contains('External power ref'), 1, 0)
df_logs_join.epr = df_logs_join.groupby(['Timestamp', 'Turbine_ID'])[['epr']].transform('max')
df_logs_join = df_logs_join.groupby(['Timestamp', 'Turbine_ID','epr'])[['logs_Remark']].agg(lambda x: ', '.join(x.unique())).reset_index()

# merge stops with signals
assert df_logs_join[df_logs_join.duplicated(subset=['Timestamp','Turbine_ID'], keep=False)].shape[0] == 0, 'Duplicates in Timestamp, Turbine_ID'
df = df.merge(df_logs_join, how='left', on=['Timestamp','Turbine_ID'])
df.epr = df.epr.fillna(0)

In [28]:
# adjust External Power Ref flag
df.epr = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2016-07-22T15:50:00.000'), 1, df.epr)
df.epr = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2016-10-02T18:00:00.000'), 1, df.epr)
df.epr = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2017-03-26T03:00:00.000'), 1, df.epr)
df.epr = np.where((df.Turbine_ID=='T07')&(df.Timestamp=='2016-01-01T00:00:00.000'), 1, df.epr)
df.epr = np.where((df.Turbine_ID=='T07')&(df.Timestamp=='2017-03-26T03:00:00.000'), 1, df.epr)
df.epr = np.where((df.Turbine_ID=='T11')&(df.Timestamp=='2017-03-26T03:00:00.000'), 1, df.epr)

In [29]:
# calculate on/off indicator
df['on'] = np.where(df.epr==1, 1, np.nan)

# chunks of 'External power ref:' each 40 minutes
df.on = np.where((df.groupby(['Turbine_ID']).epr.shift(-1)==1)&(df.groupby(['Turbine_ID']).epr.shift(2)==1), 1, df.on)
df.on = np.where((df.groupby(['Turbine_ID']).epr.shift(-2)==1)&(df.groupby(['Turbine_ID']).epr.shift(1)==1), 1, df.on)

# ending epr block and still giving power for 20min
df.on = np.where((df.on.isna())&(df.groupby(['Turbine_ID']).epr.shift(1)==1)&(df.Grd_Prod_Pwr_Avg>=0), 1, df.on)
df.on = np.where((df.on.isna())&(df.groupby(['Turbine_ID']).epr.shift(2)==1)&(df.Grd_Prod_Pwr_Avg>=0), 1, df.on)

# (on, nan, on) AND (Generator In) AND (epr in previous row)
df.on = np.where((df.on.isna())&(df.groupby(['Turbine_ID']).on.shift(1)==1)&(df.groupby(['Turbine_ID']).on.shift(-1)==1)&(df.groupby(['Turbine_ID']).epr.shift(1)==1), 1, df.on)

# (on, off, on) AND producing power
for i in range(10):
    df.on = np.where((df.on.isna())&(df.groupby(['Turbine_ID']).on.shift(1)==1)&(df.groupby(['Turbine_ID']).on.shift(1)==1)&(df.Grd_Prod_Pwr_Avg>0), 1, df.on)

# df.on = np.where(df.groupby(['Turbine_ID']).epr.shift(-1)==1, 1, df.on)

### Merge signals & failures

In [30]:
# concat logs
df_fail_join = df_fail.copy()

# convert to 10 min
df_fail_join.Timestamp = pd.to_datetime(df_fail_join.Timestamp.dt.strftime('%Y-%m-%d %H:%M').str[:-1] + '0:00')
df_fail_join.Remark = df_fail_join.Remarks.fillna('').astype(str)
df_fail_join['fail_Remark'] = df_fail_join.Component + ': ' + df_fail_join.Remarks.fillna('').astype(str)
df_fail_join = df_fail_join.groupby(['Timestamp', 'Turbine_ID'])[['fail_Remark']].agg(lambda x: ', '.join(x.unique())).reset_index()

# merge fails with signals
assert df_fail_join[df_fail_join.duplicated(subset=['Timestamp','Turbine_ID'], keep=False)].shape[0] == 0, 'Duplicates in Timestamp, Turbine_ID'
df = df.merge(df_fail_join, how='left', on=['Timestamp','Turbine_ID'])

/tmp/ipykernel_1406/3302004226.py:6: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_fail_join.Remark = df_fail_join.Remarks.fillna('').astype(str)


In [31]:
# add fail indicator (from fail to epr)
df['fail'] = np.where(df.groupby(['Turbine_ID']).fail_Remark.shift(1).isna()==False, 1, 0)

# adjust fail
df.fail = np.where((df.Turbine_ID=='T01')&(df.Timestamp>='2016-07-17T14:10:00.000')&(df.Timestamp<='2016-07-19T12:40:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2016-04-04T18:40:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T06')&(df.Timestamp>='2016-07-10T20:50:00.000')&(df.Timestamp<='2016-07-22T15:40:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2016-10-27T17:30:00.000'), 1, df.fail)
df.fail_Remark = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2016-10-27T17:30:00.000'), 'GENERATOR: Generator replaced', df.fail_Remark)
df.fail = np.where((df.Turbine_ID=='T06')&(df.Timestamp>='2017-08-19T02:20:00.000')&(df.Timestamp<='2017-08-19T12:50:00.000'), 1, df.fail)
df.fail_Remark = np.where((df.Turbine_ID=='T06')&(df.Timestamp=='2017-10-17T08:50:00.000'), 'GEARBOX: Gearbox bearings damaged', df.fail_Remark)
df.fail = np.where((df.Turbine_ID=='T07')&(df.Timestamp>='2016-04-29T18:20:00.000')&(df.Timestamp<='2016-04-30T14:00:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T07')&(df.Timestamp>='2016-07-10T09:50:00.000')&(df.Timestamp<='2016-07-10T18:50:00.000'), 1, df.fail)
df.fail_Remark = np.where((df.Turbine_ID=='T07')&(df.Timestamp=='2016-07-10T04:00:00.000'), 'TRANSFORMER: High temperature transformer', df.fail_Remark)
df.fail = np.where((df.Turbine_ID=='T07')&(df.Timestamp>='2016-08-22T13:50:00.000')&(df.Timestamp<='2016-08-23T03:50:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T07')&(df.Timestamp=='2017-06-16T17:40:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T07')&(df.Timestamp=='2017-10-19T07:40:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T11')&(df.Timestamp>='2016-03-03T08:00:00.000')&(df.Timestamp<='2016-03-04T12:10:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T11')&(df.Timestamp>='2016-10-17T04:20:00.000')&(df.Timestamp<='2016-10-18T13:20:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T11')&(df.Timestamp>='2017-04-26T06:40:00.000')&(df.Timestamp<='2017-04-26T18:50:00.000'), 1, df.fail)
df.fail = np.where((df.Turbine_ID=='T11')&(df.Timestamp>='2017-09-12T07:40:00.000')&(df.Timestamp<='2017-09-12T18:20:00.000'), 1, df.fail)

# forward fill fail indicator
for i in range(1200):
    df.fail = np.where((df.groupby(['Turbine_ID']).fail.shift(1)==1)&(df.Grd_Prod_Pwr_Avg.fillna(0)<=0), 1, df.fail)

# remove fail indicator at starts of fail with power > 0
for i in range(2):
    df.fail = np.where((df.fail==1)&(df.groupby(['Turbine_ID']).fail.shift(1)!=1)&(df.Grd_Prod_Pwr_Avg>0), 0, df.fail)

Average windspeed of on/off blocks to isolate offs due to low or high speed (<4m/s, >25m/s)
- Add `block_id`, identificator of each operation block (`on`, `off`,  `off - low wind` or `off - high wind` consecutive timestamps)
- Add `block_len`, length of each block_id

In [34]:
# on/off block id
df['block_id'] = df.on.fillna(0)
df.block_id = np.where(df.groupby(['Turbine_ID']).block_id.shift(1)!=df.block_id, 1, 0)
df.block_id = df.groupby(['Turbine_ID']).block_id.transform('cumsum')

# identify blocks off by low wind
df['avg_block_windspeed'] = df.groupby(['Turbine_ID','block_id']).Amb_WindSpeed_Avg.transform('mean')
df['low_wind_stop'] = np.where((df.on.isna())&(df.avg_block_windspeed<4), 1, np.where(df.on.isna(), 0, np.nan))
df['high_wind_stop'] = np.where((df.on.isna())&(df.avg_block_windspeed>25), 1, np.where(df.on.isna(), 0, np.nan))

# generate status
df['status'] = np.where(df.fail==1, 'failure', np.where(df.on==1, 'on', np.where(df.low_wind_stop==1, 'off - low wind', np.where(df.high_wind_stop==1, 'off - high wind', 'off'))))

# extend status off
for i in range(100):
    df.status = np.where((df.groupby(['Turbine_ID']).status.shift(1)=='off')&(df.Grd_Prod_Pwr_Avg.fillna(0)<=0), 'off', df.status)
    df.status = np.where((df.groupby(['Turbine_ID']).status.shift(1)=='off - low wind')&(df.Grd_Prod_Pwr_Avg.fillna(0)<=0)&(df.Amb_WindSpeed_Avg.fillna(0)<=4), 'off - low wind', df.status)

# update block_id with failures
df['block_id'] = np.where(df.groupby(['Turbine_ID']).status.shift(1)!=df.status, 1, 0)
df.block_id = df.groupby(['Turbine_ID']).block_id.transform('cumsum')
df['block_len'] = df.groupby(['Turbine_ID','block_id']).block_id.transform('count')

df.drop(columns=['epr','on','avg_block_windspeed','low_wind_stop','high_wind_stop'], inplace=True)
df.head()

,Turbine_ID,Timestamp,Gen_RPM_Max,Gen_RPM_Min,Gen_RPM_Avg,Gen_RPM_Std,Gen_Bear_Temp_Avg,Gen_Phase1_Temp_Avg,Gen_Phase2_Temp_Avg,Gen_Phase3_Temp_Avg,Hyd_Oil_Temp_Avg,Gear_Oil_Temp_Avg,Gear_Bear_Temp_Avg,Nac_Temp_Avg,Rtr_RPM_Max,Rtr_RPM_Min,Rtr_RPM_Avg,Amb_WindSpeed_Max,Amb_WindSpeed_Min,Amb_WindSpeed_Avg,Amb_WindSpeed_Std,Amb_WindDir_Relative_Avg,Amb_WindDir_Abs_Avg,Amb_Temp_Avg,Prod_LatestAvg_ActPwrGen0,Prod_LatestAvg_ActPwrGen1,Prod_LatestAvg_ActPwrGen2,Prod_LatestAvg_TotActPwr,Prod_LatestAvg_ReactPwrGen0,Prod_LatestAvg_ReactPwrGen1,Prod_LatestAvg_ReactPwrGen2,Prod_LatestAvg_TotReactPwr,HVTrafo_Phase1_Temp_Avg,HVTrafo_Phase2_Temp_Avg,HVTrafo_Phase3_Temp_Avg,Grd_InverterPhase1_Temp_Avg,Cont_Top_Temp_Avg,Cont_Hub_Temp_Avg,Cont_VCP_Temp_Avg,Gen_SlipRing_Temp_Avg,Spin_Temp_Avg,Blds_PitchAngle_Min,Blds_PitchAngle_Max,Blds_PitchAngle_Avg,Blds_PitchAngle_Std,Cont_VCP_ChokcoilTemp_Avg,Grd_RtrInvPhase1_Temp_Avg,Grd_RtrInvPhase2_Temp_Avg,Grd_RtrInvPhase3_Temp_Avg,Cont_VCP_WtrTemp_Avg,Grd_Prod_Pwr_Avg,Grd_Prod_CosPhi_Avg,Grd_Prod_Freq_Avg,Grd_Prod_VoltPhse1_Avg,Grd_Prod_VoltPhse2_Avg,Grd_Prod_VoltPhse3_Avg,Grd_Prod_CurPhse1_Avg,Grd_Prod_CurPhse2_Avg,Grd_Prod_CurPhse3_Avg,Grd_Prod_Pwr_Max,Grd_Prod_Pwr_Min,Grd_Busbar_Temp_Avg,Rtr_RPM_Std,Amb_WindSpeed_Est_Avg,Grd_Prod_Pwr_Std,Grd_Prod_ReactPwr_Avg,Grd_Prod_ReactPwr_Max,Grd_Prod_ReactPwr_Min,Grd_Prod_ReactPwr_Std,Grd_Prod_PsblePwr_Avg,Grd_Prod_PsblePwr_Max,Grd_Prod_PsblePwr_Min,Grd_Prod_PsblePwr_Std,Grd_Prod_PsbleInd_Avg,Grd_Prod_PsbleInd_Max,Grd_Prod_PsbleInd_Min,Grd_Prod_PsbleInd_Std,Grd_Prod_PsbleCap_Avg,Grd_Prod_PsbleCap_Max,Grd_Prod_PsbleCap_Min,Grd_Prod_PsbleCap_Std,Gen_Bear2_Temp_Avg,Nac_Direction_Avg,logs_Remark,fail_Remark,fail,block_id,status,block_len
0,T01,2016-01-01 00:00:00,1277.4,1226.1,1249.0,9.0,41.0,58.0,59.0,58.0,30.0,44.0,48.0,28.0,11.3,10.9,11.1,11.6,0.5,3.3,0.9,-12.4,206.1,18.0,-107.0,4420.0,0.0,4313.0,-99.0,-5636.0,0.0,-5735.0,68.0,76.0,65.0,39.0,39.0,28.0,43.0,25.0,20.0,-1.1,4.5,0.6,0.9,91.0,39.0,39.0,38.0,39.0,26.2,0.7,50.0,401.6,399.9,399.2,46.5,61.1,45.5,118.4,-33.5,38.0,0.1,3.6,35.3,-34.9,13.2,-100.0,36.2,29.4,119.3,0.0,31.9,-144.4,0.0,-584.5,157.1,144.4,584.5,0.0,157.1,37.0,218.5,NaN,NaN,0,1,off - low wind,1
1,T01,2016-01-01 00:10:00,1268.3,93.3,999.7,435.9,41.0,58.0,59.0,59.0,30.0,44.0,48.0,28.0,11.2,0.0,8.8,12.5,0.4,3.2,0.9,-1.1,217.4,18.0,-753.0,2488.0,0.0,1735.0,-867.0,-3871.0,0.0,-4738.0,68.0,76.0,65.0,38.0,39.0,28.0,42.0,25.0,20.0,-0.9,59.6,8.1,14.7,91.0,37.0,36.0,37.0,37.0,10.1,0.7,50.0,401.9,400.7,399.5,34.3,41.9,33.6,101.5,-38.2,38.0,4.0,3.3,30.2,-28.0,1.8,-100.1,34.6,21.7,102.3,0.0,24.7,-79.6,0.0,-501.0,123.7,79.6,501.0,0.0,123.7,37.0,218.5,"Generator 1 in, External power ref.:2000kW",NaN,0,2,on,13
2,T01,2016-01-01 00:20:00,1394.4,220.3,774.0,486.1,41.0,57.0,57.0,57.0,30.0,43.0,46.0,29.0,12.3,1.9,6.8,8.8,0.7,4.3,1.0,-5.8,207.4,18.0,-784.0,10488.0,0.0,9704.0,-410.0,-1865.0,0.0,-2275.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-2.2,24.0,10.4,12.4,87.0,38.0,37.0,37.0,38.0,59.0,0.7,50.0,401.4,400.5,398.8,78.6,92.7,86.1,219.4,-34.1,38.0,4.3,4.1,77.5,-13.9,999.9,-108.1,154.8,88.9,224.9,0.0,54.3,-314.0,0.0,-1000.0,357.4,314.0,1000.0,0.0,357.4,37.0,213.3,NaN,NaN,0,2,on,13
3,T01,2016-01-01 00:30:00,1306.6,1212.4,1257.1,17.0,40.0,56.0,57.0,57.0,30.0,44.0,48.0,28.0,11.5,10.7,11.1,9.9,0.7,4.4,1.1,9.6,232.0,18.0,0.0,22673.0,0.0,22673.0,0.0,-15822.0,0.0,-15822.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-1.9,0.9,-0.5,0.8,87.0,38.0,38.0,38.0,38.0,135.7,0.8,50.0,399.7,398.1,396.9,128.1,160.8,136.8,250.3,-27.7,38.0,0.1,4.6,53.9,-94.9,0.0,-105.1,18.4,135.3,261.4,0.0,53.1,-654.0,0.0,-1000.0,245.8,654.0,1000.0,0.0,245.8,36.0,222.4,NaN,NaN,0,2,on,13
4,T01,2016-01-01 00:40:00,1320.9,1224.1,1257.7,18.0,40.0,57.0,58.0,57.0,30.0,44.0,48.0,28.0,11.7,10.8,11.1,9.9,0.8,4.1,1.2,23.3,245.6,18.0,-75.0,16581.0,0.0,16506.0,-79.0,-13314.0,0.0,-13394.0,68.0,76.0,65.0,39.0,39.0,28.0,42.0,25.0,20.0,-2.0,4.4,-0.1,1.2,88.0,39.0,38.0,38.0,39.0,98.9,0.7,50.0,399.5,397.9,396.8,104.7,130

In [35]:
status = df.groupby(['Turbine_ID','status']).agg(generated_power=('Grd_Prod_Pwr_Avg','sum'), n_blocks=('block_id','nunique'), total_duration=('status','count')).reset_index()
status.total_duration = status.total_duration*10
status['mean_duration'] = status.total_duration / status.n_blocks
status.sort_values(by=['status'])

,Turbine_ID,status,generated_power,n_blocks,total_duration,mean_duration
0,T01,failure,14210.00,2,2880,1440.000000
4,T06,failure,-2559.65,8,29620,3702.500000
8,T07,failure,22907.70,6,16620,2770.000000
12,T11,failure,-605.80,4,5080,1270.000000
1,T01,off,-568.60,11,2160,196.363636
5,T06,off,-843.40,11,3010,273.636364
9,T07,off,-726.00,17,2720,160.000000
13,T11,off,-420.60,13,2340,180.000000
2,T01,off - low wind,-168824.90,1351,293210,217.031828
6,T06,off - low wind,-168738.90,1364,277590,203.511730


### Indicators of moments prior to a shutdown
- Add `off_start`, indicator fot the moment when a turbine stops (for a minimum duration)
- Add `prior_7d_off`, indicator for the 8th previous day to a stop

In [36]:
df['off_start'] = np.where((df.status=='off')&(df.groupby(['Turbine_ID']).status.shift(1) != 'off')&(df.block_len>3), 1, 
                  np.where((df.status=='failure')&(df.groupby(['Turbine_ID']).status.shift(1) != 'failure')&(df.block_len>3), 1, 0))

# status of machines the 8th previous day to a stop
df['prior_7d_off'] = 0
for i in range(0, 6*24):
    df.prior_7d_off = np.where(df.groupby(['Turbine_ID']).status.shift(-1*6*24 - i) == 'off', 1, df.prior_7d_off)


df['prior_1h_off'] = 0
for i in range(0, 6):
    df.prior_1h_off = np.where(df.groupby(['Turbine_ID']).status.shift(-i) == 'off', 1, df.prior_1h_off)

In [89]:
df.to_csv('./data/df_clean.csv', index=None)

### Backup

In [22]:
check = df[['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','epr','on','low_wind_stop','high_wind_stop','status','block_id','logs_Remark','fail_Remark']]

In [16]:
df[df.status=='failure'].groupby(['Turbine_ID','block_id']).Grd_Prod_Pwr_Avg.sum().sort_values(ascending=False)

Turbine_ID  block_id
T07         952         18965.80
T01         869         14210.00
T07         812          8332.40
T11         2494          642.70
T01         2427            0.00
T11         1896          -12.60
T06         1179          -17.50
            1055          -24.90
            945           -61.20
            545           -69.20
            2695         -100.70
T07         580          -119.90
            2594         -134.50
T06         935          -236.10
            2513         -329.90
T11         260          -525.20
            1188         -710.70
T07         2118        -1081.70
T06         1287        -1720.15
T07         2428        -3054.40
Name: Grd_Prod_Pwr_Avg, dtype: float64

In [76]:
df[(df.status=='off')&(df.Grd_Prod_Pwr_Avg>0)][['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','status','block_id','logs_Remark','fail_Remark']]

,Turbine_ID,Timestamp,Grd_Prod_Pwr_Avg,Amb_WindSpeed_Avg,status,block_id,logs_Remark,fail_Remark
289807,T07,2017-07-04 13:10:00,16.1,10.5,off,2172,"Pause pressed on keyboard, Pause -5.1kW 3...",NaN


In [63]:
# check = df[['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','logs_Remark','fail_Remark','fail']]
df[(df.fail_Remark.isna()==False)][['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','logs_Remark','fail_Remark','fail']]

,Turbine_ID,Timestamp,Grd_Prod_Pwr_Avg,Amb_WindSpeed_Avg,logs_Remark,fail_Remark,fail
28669,T01,2016-07-18 02:10:00,441.6,12.5,"Run, Reset:2016-07-18 02:20:00Thermoerr. oil p...",GEARBOX: Gearbox pump damaged,0
84751,T01,2017-08-11 13:10:00,1999.8,16.4,"Key switch: Local/Remote, User 0 primary ac...",TRANSFORMER: Transformer fan damaged,0
118913,T06,2016-04-04 18:50:00,-21.6,9.7,"Key switch: Local/Remote, User 0 primary ac...",HYDRAULIC_GROUP: Error in pitch regulation,0
133030,T06,2016-07-11 19:40:00,NaN,NaN,NaN,GENERATOR: Generator replaced,0
134886,T06,2016-07-24 17:00:00,1999.4,14.8,"User 0 primary access, Key switch: Local co...",GENERATOR: Generator temperature sensor failure,0
140880,T06,2016-09-04 08:00:00,227.6,5.3,Reset:2016-09-04 08:10:00High temperature Gen2...,GENERATOR: High temperature generator error,0
144966,T06,2016-10-02 17:00:00,237.7,5.2,"Key switch: Local control, Key switch: Local/R...",GENERATOR: Refrigeration system and temperatur...,0
148562,T06,2016-10-27 16:20:00,1942.2,12.2,External power ref.:2000kW,GENERATOR: Generator replaced,0
148569,T06,2016-10-27 17:30:00,NaN,NaN,NaN,GENERATOR: Generator replaced,0
191146,T06,2017-08-19 09:40:00,-5.3,1.6,"Nac.vent.2, nac/gear: 33/ 41°C, Nac.vent.1, na...",HYDRAULIC_GROUP: Oil leakage in Hub,0


In [15]:
df[(df.fail_Remark.isna()==False)][['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','status','block_id','logs_Remark','fail_Remark', 'fail']]
# check = df[['Turbine_ID','Timestamp','Grd_Prod_Pwr_Avg','Amb_WindSpeed_Avg','status','block_id','logs_Remark','fail_Remark', 'fail']]

,Turbine_ID,Timestamp,Grd_Prod_Pwr_Avg,Amb_WindSpeed_Avg,status,block_id,logs_Remark,fail_Remark,fail
28669,T01,2016-07-18 02:10:00,441.6,12.5,on,868,"Run, Reset:2016-07-18 02:20:00Thermoerr. oil p...",GEARBOX: Gearbox pump damaged,0
84751,T01,2017-08-11 13:10:00,1999.8,16.4,on,2426,"Key switch: Local/Remote, User 0 primary ac...",TRANSFORMER: Transformer fan damaged,0
118913,T06,2016-04-04 18:50:00,-21.6,9.7,on,544,"Key switch: Local/Remote, User 0 primary ac...",HYDRAULIC_GROUP: Error in pitch regulation,0
133030,T06,2016-07-11 19:40:00,NaN,NaN,off - low wind,933,NaN,GENERATOR: Generator replaced,0
134886,T06,2016-07-24 17:00:00,1999.4,14.8,on,942,"User 0 primary access, Key switch: Local co...",GENERATOR: Generator temperature sensor failure,0
140880,T06,2016-09-04 08:00:00,227.6,5.3,on,1050,Reset:2016-09-04 08:10:00High temperature Gen2...,GENERATOR: High temperature generator error,0
144966,T06,2016-10-02 17:00:00,237.7,5.2,on,1172,"Key switch: Local control, Key switch: Local/R...",GENERATOR: Refrigeration system and temperatur...,0
148562,T06,2016-10-27 16:20:00,1942.2,12.2,on,1278,External power ref.:2000kW,GENERATOR: Generator replaced,0
191146,T06,2017-08-19 09:40:00,-5.3,1.6,off - low wind,2507,"Nac.vent.2, nac/gear: 33/ 41°C, Nac.vent.1, na...",HYDRAULIC_GROUP: Oil leakage in Hub,0
199635,T06,2017-10-17 08:30:00,717.5,7.2,on,2690,"Key switch: Local/Remote, User 0 primary ac...",GEARBOX: Gearbox bearings damaged,0
